In [1]:
from game import Game

from fiftyone import ViewField as F

import torch

import pandas as pd

from memory import FastAttractor

from strategies import FastAttractorMemory

from tqdm import tqdm

import math

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns



In [2]:
n_trials = 20

size_range = [[3,3], [4,4], [5,5], [6,6], [7,7]]

steps = 10

In [3]:
game = Game(
            dataset_name='coco-2017', 
            split='validation', 
            field="ground_truth", 
            ds_filter=F("supercategory").is_in(('person', 'animal', 'food', 'vehicle', 'indoor')),
            dataset_dir ="/home/ravi/datasets/coco-2017",
        )

Found annotations at '/home/ravi/datasets/coco-2017/raw/instances_val2017.json'
Images already downloaded
Existing download of split 'validation' is sufficient
Loading existing dataset 'coco-2017-validation'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [4]:
def play_strategy(strategy, game, max_steps=500):
    win = False
    steps = 0
    while not win and steps < max_steps:
        strategy.pick()
        win = game.check_win()
        torch.cuda.empty_cache()
        steps+=1


In [5]:
memory = FastAttractor(14, 0.8, 0.85, f='tanh')

In [6]:
clicks = [[0, 0], [4,2], [4,8],[3,3],[2,1],[3,8], [3,3], [0, 5], [1,1]]
clicks += [[0,0], [3,3], [0, 5]]

In [7]:
for c in clicks:
    curr_oh = torch.full([5], -1.0)
    curr_oh[c[0]] = 1.0
    pos_oh = torch.full([9], -1.0)
    pos_oh[c[1]] = 1.0
    h = torch.unsqueeze(torch.concat([pos_oh, curr_oh]), dim=0)
    memory(h, 10)

In [8]:
print(memory.M[9+0])

tensor([-0.1196, -0.1361, -0.1446, -0.1295, -0.1446,  0.0871, -0.1446, -0.1446,
        -0.1436,  0.1165, -0.1366, -0.1442, -0.1281, -0.1445],
       grad_fn=<SelectBackward0>)


In [9]:
pos_oh = torch.zeros(9)
curr_oh = torch.zeros(5)
curr_oh[0] = 2.
x = torch.unsqueeze(torch.concat([pos_oh, curr_oh]), dim=0)
memory(x, 10)

tensor([[-0.1511, -0.1724, -0.1940, -0.1019, -0.1941,  0.0175, -0.1941, -0.1941,
         -0.1916,  0.6592, -0.1737, -0.1930, -0.0986, -0.1937]])